# Taller 4 TSP Ciudades

## Imports

In [3]:
import argparse
import math
import unicodedata
from typing import Dict, List, Tuple, Iterable
import pandas as pd
import numpy as np
import sys

## Carga de datos

In [4]:
def detect_header_row(xlsx_path: str, max_search: int = 30) -> int:
    raw = pd.read_excel(xlsx_path, header=None)
    for i in range(min(max_search, len(raw))):
        row = raw.iloc[i].astype(str).str.upper().tolist()
        if any('LATITUD' == x for x in row) and any('LONGITUD' == x for x in row):
            return i
    return 0

def load_points_flexible(xlsx_path: str) -> pd.DataFrame:
    """Devuelve DataFrame con columnas: lat, lon, name, name_norm (si name existe)."""
    hdr = detect_header_row(xlsx_path)
    df = pd.read_excel(xlsx_path, header=hdr)
    df.columns = [str(c).strip().upper() for c in df.columns]

    lat_col_candidates = [c for c in df.columns if 'LAT' in c]
    lon_col_candidates = [c for c in df.columns if 'LON' in c]
    if not lat_col_candidates or not lon_col_candidates:
        raise ValueError("No se encontraron columnas de latitud/longitud.")

    lat_col = lat_col_candidates[0]
    lon_col = lon_col_candidates[0]

    if 'NOMBRE.1' in df.columns:
        name_col = 'NOMBRE.1'
    elif 'MUNICIPIO' in df.columns:
        name_col = 'MUNICIPIO'
    elif 'NOMBRE' in df.columns:
        name_col = 'NOMBRE'
    elif 'POBLADO' in df.columns:
        name_col = 'POBLADO'
    else:
        name_col = None

    if name_col is None:
        tmp = df[[lat_col, lon_col]].dropna().copy()
        tmp['name'] = [f"NODO_{i}" for i in range(len(tmp))]
        clean = tmp.rename(columns={lat_col:'lat', lon_col:'lon'})
        clean['name_norm'] = clean['name']
        return clean[['lat','lon','name','name_norm']]

    clean = df[[lat_col, lon_col, name_col]].dropna().rename(
        columns={lat_col:'lat', lon_col:'lon', name_col:'name'}
    ).reset_index(drop=True)

    def strip_accents(s: str) -> str:
        return ''.join(c for c in unicodedata.normalize('NFD', str(s)) if unicodedata.category(c) != 'Mn')

    clean['name_norm'] = clean['name'].apply(lambda s: strip_accents(s).upper().strip())
    return clean[['lat','lon','name','name_norm']]

def load_points_rigida(xlsx_path: str) -> pd.DataFrame:
    """Versión 'rígida' que espera LAT*, LON* y opcionalmente NOMBRE/MUNICIPIO/POBLADO."""
    hdr = detect_header_row(xlsx_path)
    df = pd.read_excel(xlsx_path, header=hdr)
    df.columns = [str(c).strip().upper() for c in df.columns]
    name_col_candidates = [c for c in df.columns if 'NOMBRE' in c or 'MUNICIPIO' in c or 'POBLADO' in c]
    lat_col_candidates = [c for c in df.columns if 'LAT' in c]
    lon_col_candidates = [c for c in df.columns if 'LON' in c]
    if not lat_col_candidates or not lon_col_candidates:
        raise ValueError("No se encontraron columnas de LAT/LON.")
    name_col = name_col_candidates[0] if name_col_candidates else None
    lat_col = lat_col_candidates[0]
    lon_col = lon_col_candidates[0]
    keep = [c for c in [name_col, lat_col, lon_col] if c is not None]
    clean = df[keep].dropna()
    rename_map = {}
    if name_col: rename_map[name_col] = 'name'
    rename_map[lat_col] = 'lat'
    rename_map[lon_col] = 'lon'
    clean = clean.rename(columns=rename_map).reset_index(drop=True)

    if 'name' in clean.columns:
        def strip_accents(s: str) -> str:
            return ''.join(c for c in unicodedata.normalize('NFD', str(s)) if unicodedata.category(c) != 'Mn')
        clean['name_norm'] = clean['name'].apply(lambda s: strip_accents(s).upper().strip())
    else:
        clean['name'] = [f"NODO_{i}" for i in range(len(clean))]
        clean['name_norm'] = clean['name']
    return clean[['lat','lon','name','name_norm']]

## Algoritmo TSP

In [5]:
def haversine_km(lat1, lon1, lat2, lon2) -> float:
    R = 6371.0088
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*(math.sin(dlambda/2)**2)
    return 2*R*math.asin(math.sqrt(a))

def distance_matrix(coords: List[Tuple[float,float]]) -> np.ndarray:
    n = len(coords)
    D = np.zeros((n,n), dtype=float)
    for i in range(n):
        for j in range(i+1, n):
            d = haversine_km(coords[i][0], coords[i][1], coords[j][0], coords[j][1])
            D[i,j] = D[j,i] = d
    return D

def held_karp(D: np.ndarray) -> Tuple[float, List[int]]:
    n = D.shape[0]
    dp: Dict[Tuple[int,int], Tuple[float,int]] = {(1<<0, 0): (0.0, -1)} 
    for r in range(2, n+1):
        for subset in (mask for mask in range(1<<n) if (mask & 1) and (mask.bit_count()==r)):
            for j in range(1, n):
                if not (subset & (1<<j)): 
                    continue
                best = (float("inf"), -1)
                prevmask = subset ^ (1<<j)
                for k in range(n):
                    if k==j or not (prevmask & (1<<k)):
                        continue
                    if (prevmask, k) in dp:
                        cand = dp[(prevmask,k)][0] + D[k,j]
                        if cand < best[0]:
                            best = (cand, k)
                if best[0] < float("inf"):
                    dp[(subset, j)] = best

    full = (1<<n) - 1
    best = (float("inf"), -1)
    for j in range(1, n):
        if (full, j) in dp:
            cand = dp[(full, j)][0] + D[j, 0]
            if cand < best[0]:
                best = (cand, j)

    tour = [0]
    mask = full
    j = best[1]
    while j != -1:
        tour.append(j)
        mask, j = mask ^ (1<<j), dp[(mask, tour[-1])][1]
    tour.reverse()
    tour = tour[:-1] 
    return best[0], tour

def mst_prim(D: np.ndarray) -> List[Tuple[int,int]]:
    n = len(D)
    in_mst = [False]*n
    in_mst[0] = True
    edges = []
    while len(edges) < n-1:
        best = (float("inf"), -1, -1)
        for u in range(n):
            if not in_mst[u]: 
                continue
            for v in range(n):
                if in_mst[v]: 
                    continue
                if D[u,v] < best[0]:
                    best = (D[u,v], u, v)
        _, u, v = best
        edges.append((u,v))
        in_mst[v] = True
    return edges

def degree_from_edges(n: int, edges: Iterable[Tuple[int,int]]) -> List[int]:
    deg = [0]*n
    for u,v in edges:
        deg[u]+=1; deg[v]+=1
    return deg

def greedy_min_matching(nodes: List[int], D: np.ndarray) -> List[Tuple[int,int]]:
    nodes = list(nodes)
    used = set()
    match = []
    pairs = []
    for i in range(len(nodes)):
        for j in range(i+1, len(nodes)):
            u, v = nodes[i], nodes[j]
            pairs.append((D[u,v], u, v))
    for _,u,v in sorted(pairs):
        if u in used or v in used:
            continue
        used.add(u); used.add(v)
        match.append((u,v))
    return match

def eulerian_tour(n: int, edges: List[Tuple[int,int]]) -> List[int]:
    adj = {i: [] for i in range(n)}
    for idx,(u,v) in enumerate(edges):
        adj[u].append((v, idx))
        adj[v].append((u, idx))
    used = set()
    stack = [0]
    path = []
    while stack:
        u = stack[-1]
        while adj[u] and adj[u][-1][1] in used:
            adj[u].pop()
        if not adj[u]:
            path.append(stack.pop())
        else:
            v, eid = adj[u].pop()
            if eid in used:
                continue
            used.add(eid)
            stack.append(v)
    return path[::-1]

def shortcut_to_hamiltonian(path: List[int]) -> List[int]:
    seen = set()
    tour = []
    for v in path:
        if v not in seen:
            seen.add(v)
            tour.append(v)
    tour.append(tour[0])
    return tour[:-1]

def two_opt(route: List[int], D: np.ndarray) -> Tuple[float, List[int]]:
    n = len(route)
    improved = True
    while improved:
        improved = False
        for i in range(1, n-2):
            for j in range(i+1, n-1):
                a, b = route[i-1], route[i]
                c, d = route[j], route[(j+1)%n]
                delta = (D[a,c] + D[b,d]) - (D[a,b] + D[c,d])
                if delta < -1e-9:
                    route[i:j+1] = reversed(route[i:j+1])
                    improved = True
    length = sum(D[route[i], route[(i+1)%n]] for i in range(n))
    return float(length), route

def christofides_2opt(D: np.ndarray) -> Tuple[float, List[int]]:
    n = len(D)
    mst = mst_prim(D)
    deg = degree_from_edges(n, mst)
    odd = [i for i,d in enumerate(deg) if d%2==1]
    matching = greedy_min_matching(odd, D) 
    edges = mst + matching
    path = eulerian_tour(n, edges)
    tour = shortcut_to_hamiltonian(path)
    length = sum(D[tour[i], tour[(i+1)%n]] for i in range(n))
    length, tour = two_opt(tour, D)
    return float(length), tour


def _ru_maxrss_bytes() -> int:
    try:
        import resource
        ru = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
        return int(ru if sys.platform == "darwin" else ru * 1024)
    except Exception:
        return 0

def measure_time_and_memory(fn, *args, **kwargs):
    import time, tracemalloc
    tracemalloc.start()
    t0 = time.perf_counter()
    result = fn(*args, **kwargs)
    t1 = time.perf_counter()
    _, peak_traced = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    metrics = {
        "time_s": float(t1 - t0),
        "peak_tracemalloc_bytes": int(peak_traced),
    }
    return result, metrics

def fmt_bytes(n: int) -> str:
    for unit in ["B","KB","MB","GB","TB"]:
        if abs(n) < 1024.0:
            return f"{n:.2f} {unit}"
        n /= 1024.0
    return f"{n:.2f} PB"

def normalize(s: str) -> str:
    return ''.join(c for c in unicodedata.normalize('NFD', str(s)) if unicodedata.category(c) != 'Mn').upper().strip()

def pick_by_names(df: pd.DataFrame, names: List[str]) -> pd.DataFrame:
    wanted_norm = [normalize(x) for x in names]
    idx_map = {n:i for i, n in enumerate(wanted_norm)}
    sub = df[df["name_norm"].isin(idx_map.keys())].copy()
    sub["order"] = sub["name_norm"].map(idx_map)
    sub = sub.sort_values("order").drop(columns=["order"]).reset_index(drop=True)
    return sub

def pick_first_n(df: pd.DataFrame, n: int, seed: int = 42) -> pd.DataFrame:
    return df.sample(n=n, random_state=seed).reset_index(drop=True)

def read_names_file(path: str) -> List[str]:
    with open(path, "r", encoding="utf-8") as f:
        return [line.strip() for line in f if line.strip()]

def tour_to_table(df: pd.DataFrame, D: np.ndarray, tour: List[int], length_km: float) -> pd.DataFrame:
    rows = []
    n = len(tour)
    for i in range(n):
        u = tour[i]
        v = tour[(i+1)%n]
        rows.append({
            "Paso": i+1,
            "Desde": df.iloc[u]["name"],
            "→": "→",
            "Hasta": df.iloc[v]["name"],
            "Distancia (km)": round(float(D[u,v]), 2)
        })
    return pd.DataFrame(rows)


## Configuración y Ejecución

In [6]:
XLSX_PATH = "../data/DIVIPOLA_Municipios.xlsx"
USE_RIGIDA = False  # True para usar load_points_rigida, False para load_points_flexible
NAMES11_FILE = None  # O ruta a archivo con nombres
NAMES15_FILE = None  # O ruta a archivo con nombres
NAMES29_FILE = None  # O ruta a archivo con nombres
SEED15 = 15
SEED29 = 29

# Cargar datos
loader = load_points_rigida if USE_RIGIDA else load_points_flexible
df = loader(XLSX_PATH)
print(f"Total de poblaciones cargadas: {len(df)}")

Total de poblaciones cargadas: 1122


### Seleccionar ciudades para cada caso

In [7]:
# Seleccionar 11 ciudades
default_11 = ["Medellín","Pereira","Bucaramanga","Cali","Leticia","Manizales",
              "Barranquilla","Cartagena","Pasto","San Andrés","Cúcuta"]
names11 = read_names_file(NAMES11_FILE) if NAMES11_FILE else default_11
df11 = pick_by_names(df, names11)
if len(df11) != 11:
    print("[Aviso] No se encontraron todas las 11 ciudades; usando muestreo.")
    df11 = pick_first_n(df, 11, seed=11)

print(f"11 ciudades seleccionadas:")
print(df11[['name', 'lat', 'lon']])

[Aviso] No se encontraron todas las 11 ciudades; usando muestreo.
11 ciudades seleccionadas:
                name       lat        lon
0         COVARACHÍA  6.519521 -72.740204
1             PÁCORA  5.500035 -75.482451
2             TOLEDO  7.225865 -72.305934
3         PAMPLONITA  7.482879 -72.635154
4      PUERTO RONDÓN  6.411762 -70.966790
5       VENTAQUEMADA  5.383385 -73.520979
6           HISPANIA  5.797039 -75.906692
7               TOCA  5.580254 -73.160697
8    HATILLO DE LOBA  8.981717 -74.090719
9             BARAYA  3.135424 -74.957978
10  PIENDAMÓ - TUNÍA  2.702420 -76.537705


In [8]:
# Seleccionar 15 ciudades
if NAMES15_FILE:
    names15 = read_names_file(NAMES15_FILE)
    df15 = pick_by_names(df, names15)
    if len(df15) != 15:
        print("[Aviso] Lista de 15 incompleta; usando muestreo.")
        df15 = pick_first_n(df, 15, seed=SEED15)
else:
    df15 = pick_first_n(df, 15, seed=SEED15)

print(f"15 ciudades seleccionadas:")
print(df15[['name', 'lat', 'lon']])

15 ciudades seleccionadas:
                name        lat        lon
0          EL CARMEN   8.867990 -73.345859
1           PINILLOS   8.886316 -74.408885
2            JAMUNDÍ   3.213892 -76.627604
3           SAN LUIS   6.024206 -75.007667
4           EL COCUY   6.353363 -72.419238
5           PAMPLONA   7.379058 -72.675903
6        PALOCABILDO   5.094036 -75.018450
7           ANAPOIMA   4.559963 -74.525695
8         VILLANUEVA  10.444913 -75.267363
9   PUERTO LEGUÍZAMO   0.059472 -75.064793
10           SOCORRO   6.462088 -73.244360
11          SANTIAGO   1.036021 -76.977517
12          TUNUNGUÁ   5.720302 -73.943309
13      EL SANTUARIO   6.124209 -75.252123
14             ISNOS   1.945591 -76.182047


In [9]:
# Seleccionar 29 ciudades
if NAMES29_FILE:
    names29 = read_names_file(NAMES29_FILE)
    df29 = pick_by_names(df, names29)
    if len(df29) != 29:
        print("[Aviso] Lista de 29 incompleta; usando muestreo.")
        df29 = pick_first_n(df, 29, seed=SEED29)
else:
    df29 = pick_first_n(df, 29, seed=SEED29)

print(f"29 ciudades seleccionadas:")
print(df29[['name', 'lat', 'lon']])

29 ciudades seleccionadas:
                   name        lat        lon
0             LA CALERA   4.700785 -73.929399
1                ACANDÍ   8.383938 -77.265603
2          SAN CAYETANO   7.847872 -72.609712
3     SAN JUAN DE ARAMA   3.289851 -73.816352
4             ALPUJARRA   3.390015 -74.940782
5               TÁMESIS   5.673759 -75.709800
6   GUADALAJARA DE BUGA   3.819854 -75.983124
7          BUENOS AIRES   2.988924 -76.621590
8               TURBACO  10.353108 -75.379715
9              ZAPATOCA   6.813726 -73.309750
10            SAN ZENÓN   9.306870 -74.358516
11              SUSACÓN   6.215201 -72.715756
12              SASAIMA   4.946805 -74.412309
13             GUATAQUÍ   4.516382 -74.782578
14  SAN JUAN NEPOMUCENO   9.967958 -75.065720
15                PIJAO   4.303979 -75.683304
16            SATIVASUR   6.083406 -72.724108
17            TAURAMENA   4.697464 -72.629224
18       PAZ DE ARIPORO   5.779915 -70.869470
19              LA CEJA   5.992525 -75.430869
20     

### Calcular matrices de distancia

In [10]:
D11 = distance_matrix(list(zip(df11["lat"], df11["lon"])))
D15 = distance_matrix(list(zip(df15["lat"], df15["lon"])))
D29 = distance_matrix(list(zip(df29["lat"], df29["lon"])))
print("Matrices de distancia calculadas.")

Matrices de distancia calculadas.


### Resolver TSP para cada caso

In [11]:
# Resolver TSP para 11 ciudades con Held-Karp
print("Resolviendo TSP para 11 ciudades...")
(len11, tour11), m11 = measure_time_and_memory(held_karp, D11)
print(f"✓ Completado: {len11:.2f} km en {m11['time_s']:.4f} s")

Resolviendo TSP para 11 ciudades...
✓ Completado: 2081.76 km en 0.2325 s


In [12]:
# Resolver TSP para 15 ciudades con Held-Karp
print("Resolviendo TSP para 15 ciudades...")
(len15, tour15), m15 = measure_time_and_memory(held_karp, D15)
print(f"✓ Completado: {len15:.2f} km en {m15['time_s']:.4f} s")

Resolviendo TSP para 15 ciudades...
✓ Completado: 2874.30 km en 7.7022 s


In [13]:
# Resolver TSP para 29 ciudades con Christofides + 2-opt
print("Resolviendo TSP para 29 ciudades...")
(len29, tour29), m29 = measure_time_and_memory(christofides_2opt, D29)
print(f"✓ Completado: {len29:.2f} km en {m29['time_s']:.4f} s")

Resolviendo TSP para 29 ciudades...
✓ Completado: 3746.46 km en 0.0256 s


### Generar tablas de resultados

In [14]:
tbl11 = tour_to_table(df11, D11, tour11, len11)
tbl15 = tour_to_table(df15, D15, tour15, len15)
tbl29 = tour_to_table(df29, D29, tour29, len29)

print("Tablas de tours generadas.")

Tablas de tours generadas.


### Visualizar tour de 11 ciudades

In [15]:
print("Tour de 11 ciudades:")
tbl11

Tour de 11 ciudades:


,Paso,Desde,→,Hasta,Distancia (km)
0,1,COVARACHÍA,→,PUERTO RONDÓN,196.31
1,2,PUERTO RONDÓN,→,TOLEDO,173.36
2,3,TOLEDO,→,PAMPLONITA,46.20
3,4,PAMPLONITA,→,HATILLO DE LOBA,231.16
4,5,HATILLO DE LOBA,→,HISPANIA,406.81
5,6,HISPANIA,→,PÁCORA,57.40
6,7,PÁCORA,→,PIENDAMÓ - TUNÍA,332.37
7,8,PIENDAMÓ - TUNÍA,→,BARAYA,181.92
8,9,BARAYA,→,VENTAQUEMADA,296.43
9,10,VENTAQUEMADA,→,TOCA,45.49


### Visualizar tour de 15 ciudades

In [16]:
print("Tour de 15 ciudades:")
tbl15

Tour de 15 ciudades:


,Paso,Desde,→,Hasta,Distancia (km)
0,1,EL CARMEN,→,VILLANUEVA,274.06
1,2,VILLANUEVA,→,PINILLOS,197.21
2,3,PINILLOS,→,EL SANTUARIO,320.89
3,4,EL SANTUARIO,→,SAN LUIS,29.23
4,5,SAN LUIS,→,PALOCABILDO,103.44
5,6,PALOCABILDO,→,JAMUNDÍ,274.87
6,7,JAMUNDÍ,→,ISNOS,149.46
7,8,ISNOS,→,SANTIAGO,134.34
8,9,SANTIAGO,→,PUERTO LEGUÍZAMO,238.79
9,10,PUERTO LEGUÍZAMO,→,ANAPOIMA,504.00


### Visualizar tour de 29 ciudades

In [17]:
print("Tour de 29 ciudades:")
tbl29

Tour de 29 ciudades:


,Paso,Desde,→,Hasta,Distancia (km)
0,1,LA CALERA,→,TAURAMENA,144.09
1,2,TAURAMENA,→,PAZ DE ARIPORO,229.03
2,3,PAZ DE ARIPORO,→,OROCUÉ,125.56
3,4,OROCUÉ,→,EL RETORNO,316.40
4,5,EL RETORNO,→,SAN JUAN DE ARAMA,216.25
5,6,SAN JUAN DE ARAMA,→,ALPUJARRA,125.31
6,7,ALPUJARRA,→,SUCRE,264.16
7,8,SUCRE,→,BUENOS AIRES,108.38
8,9,BUENOS AIRES,→,GUADALAJARA DE BUGA,116.44
9,10,GUADALAJARA DE BUGA,→,PIJAO,63.28


### Exportar resultados a CSV

In [18]:
tbl11.to_csv("../results/tour_11_ciudades.csv", index=False)
tbl15.to_csv("../results/tour_15_ciudades.csv", index=False)
tbl29.to_csv("../results/tour_29_ciudades.csv", index=False)

print("✓ Tours exportados a CSV")

✓ Tours exportados a CSV


### Resumen de métricas de rendimiento

In [19]:
metrics_rows = [
    {"case":"11", "n":len(D11), "solver":"Held-Karp", "distance_km":round(len11, 6),
     "time_s":m11["time_s"],
     "peak_tracemalloc_bytes":m11["peak_tracemalloc_bytes"]},
    {"case":"15", "n":len(D15), "solver":"Held-Karp", "distance_km":round(len15, 6),
     "time_s":m15["time_s"],
     "peak_tracemalloc_bytes":m15["peak_tracemalloc_bytes"]},
    {"case":"29", "n":len(D29), "solver":"Christofides+2opt", "distance_km":round(len29, 6),
     "time_s":m29["time_s"],
     "peak_tracemalloc_bytes":m29["peak_tracemalloc_bytes"]},
]
metrics_df = pd.DataFrame(metrics_rows)
metrics_df.to_csv("../results/metrics_tsp.csv", index=False)

print("✓ Métricas exportadas a CSV")
metrics_df

✓ Métricas exportadas a CSV


,case,n,solver,distance_km,time_s,peak_tracemalloc_bytes
0,11,11,Held-Karp,2081.760038,0.232464,795027
1,15,15,Held-Karp,2874.300620,7.702187,21252614
2,29,29,Christofides+2opt,3746.460783,0.025625,7016


### Resumen detallado

In [20]:
def show(case, length, m):
    print(f"[{case}] Distancia total: {length:.2f} km | "
          f"Tiempo: {m['time_s']:.4f} s | "
          f"Mem. Python (pico): {fmt_bytes(m['peak_tracemalloc_bytes'])}")

print("=" * 70)
print("RESUMEN DE RESULTADOS")
print("=" * 70)
show("11 (Held-Karp)", len11, m11)
show("15 (Held-Karp)", len15, m15)
show("29 (Christofides+2opt)", len29, m29)
print("=" * 70)
print("Tours: tour_11_ciudades.csv, tour_15_ciudades.csv, tour_29_ciudades.csv")
print("Métricas: metrics_tsp.csv")

RESUMEN DE RESULTADOS
[11 (Held-Karp)] Distancia total: 2081.76 km | Tiempo: 0.2325 s | Mem. Python (pico): 776.39 KB
[15 (Held-Karp)] Distancia total: 2874.30 km | Tiempo: 7.7022 s | Mem. Python (pico): 20.27 MB
[29 (Christofides+2opt)] Distancia total: 3746.46 km | Tiempo: 0.0256 s | Mem. Python (pico): 6.85 KB
Tours: tour_11_ciudades.csv, tour_15_ciudades.csv, tour_29_ciudades.csv
Métricas: metrics_tsp.csv
